In [ ]:
!pip install snowflake-connector-python
!pip install SQLAlchemy==1.4.49
!pip install ipython-sql==0.4.1

In [1]:
%load_ext sql

In [2]:
%sql snowflake://<ID>:<PASSWORD>@<ACCOUNT>/<DB>/<SCHEMA>?warehouse=<warehouse>

In [ ]:
%%sql

-- 제대로 연결되었는지 확인
select current_user(), current_role(), current_database(), current_schema();

In [ ]:
%%sql

DROP TABLE IF EXISTS yeojun.name_gender;
CREATE TABLE yeojun.name_gender(
    name varchar(32) primary key,
    gender varchar(8)
);

In [25]:
import snowflake.connector

def get_snowflake_connection():
    conn = snowflake.connector.connect(
        user=<ID>,
        password=<PASSWORD>,
        account=<ACCOUNT>,
        warehouse=<warehouse>,
        database=<DB>,
        schema=<SCHEMA>
    )
    return conn.cursor()

In [27]:
import requests

def extract(url):
    f = requests.get(url)
    return (f.text)

In [26]:
def transform(text):
    lines = text.strip().split("\n")
    records = []
    for l in lines:
        (name, gender) = l.split(",") # l = "Keeyong,M" -> [ 'keeyong', 'M' ]
        records.append([name, gender])
    return records[1:]

In [23]:
def load(records):
    """
    records = [
        [ "Keeyong", "M" ],
        [ "Claire", "F" ],
        ...
    ]
    """
    # BEGIN과 END를 사용해서 SQL 결과를 트랜잭션으로 만들어주는 것이 좋음
    cur = get_snowflake_connection()
    conn = cur.connection
    try:
        cur.execute("DELETE FROM yeojun.name_gender")

        for r in records:
            name = r[0]
            gender = r[1]
            print(name, "-", gender)
            sql = "INSERT INTO yeojun.name_gender VALUES ('{n}', '{g}')".format(n=name, g=gender)
            cur.execute(sql)
        conn.commit()
    except:
        conn.rollback()
    finally:
        conn.close()


In [37]:
link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"
data = extract(link)
data

In [ ]:
lines = transform(data)
print(len(lines))
lines[0:10]

In [ ]:
load(lines)

In [ ]:
%%sql 

SELECT gender, count(1) count
FROM yeojun.name_gender
GROUP BY gender;